In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Dataset-Capex.csv to Dataset-Capex.csv


In [3]:
CapexDS = pd.read_csv("Dataset-Capex.csv")

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from scipy.stats import randint
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

In [5]:
CapexDS[['Allocated_Budget', 'Actual_Expenditure']] = CapexDS[['Allocated_Budget', 'Actual_Expenditure']].apply(pd.to_numeric, errors='coerce')
CapexDS.fillna(0, inplace=True)

In [6]:
CapexDS['Variance'] = CapexDS['Allocated_Budget'] - CapexDS['Actual_Expenditure']
CapexDS['Budget_Status'] = np.where(CapexDS['Variance'] < 0, 'Over Budget', 'Under Budget')

In [7]:
if 'Project_Phase' in CapexDS.columns:
    encoder = LabelEncoder()
    CapexDS['Project_Phase_Encoded'] = encoder.fit_transform(CapexDS['Project_Phase'])

In [16]:
features = ['Allocated_Budget','Actual_Expenditure']
if 'Project_Phase_Encoded' in CapexDS.columns:
    features.append('Project_Phase_Encoded')

X = CapexDS[features]
y = CapexDS['Variance']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
print(X_train.head())
print(X_test.head())

    Allocated_Budget  Actual_Expenditure  Project_Phase_Encoded
55        1396207.78          1826253.94                      2
88        3291308.03          4479915.72                      0
26         821462.40          2782507.29                      1
42        2024803.44          4167167.90                      0
69        4354479.30          3882341.35                      0
    Allocated_Budget  Actual_Expenditure  Project_Phase_Encoded
83        3867191.44          1089058.57                      1
53         955109.38          4113713.16                      1
70        4022306.21           251581.39                      0
45        2188662.46          4662634.26                      2
44        4584729.12           534311.66                      2


In [19]:
rf = RandomForestRegressor(n_estimators=100, random_state=1)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

rf_mse = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE: {rf_mse}")

Random Forest MSE: 157063084311.75092


In [20]:
X_train["Allocated_Budget"] = np.log1p(X_train["Allocated_Budget"])
X_test["Allocated_Budget"] = np.log1p(X_test["Allocated_Budget"])

In [21]:
X_train["Budget_Utilization"] = X_train["Actual_Expenditure"] / X_train["Allocated_Budget"]
X_test["Budget_Utilization"] = X_test["Actual_Expenditure"] / X_test["Allocated_Budget"]

In [22]:
X_train.drop(columns=["Actual_Expenditure"], inplace=True)
X_test.drop(columns=["Actual_Expenditure"], inplace=True)

In [23]:
gb = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=1)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

gb_mse = mean_squared_error(y_test, y_pred_gb)
print(f"Gradient Boosting MSE: {gb_mse}")

Gradient Boosting MSE: 165837023381.2979


In [24]:
gb = GradientBoostingRegressor(random_state=1)
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7]
}

grid_search = GridSearchCV(gb, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

best_gb = grid_search.best_estimator_
y_pred_best = best_gb.predict(X_test)

final_mse = mean_squared_error(y_test, y_pred_best)
print(f"Optimized Gradient Boosting MSE: {final_mse}")

Optimized Gradient Boosting MSE: 166109998442.21066


In [25]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.03, max_depth=6, random_state=1)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

xgb_mse = mean_squared_error(y_test, y_pred_xgb)
print(f"XGBoost MSE: {xgb_mse}")

XGBoost MSE: 126214599184.36197


In [26]:
xgb = XGBRegressor(random_state=1)

param_grid = {
    "n_estimators": [300, 500, 700],
    "learning_rate": [0.01, 0.03, 0.05],
    "max_depth": [4, 6, 8],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_
y_pred_best_xgb = best_xgb.predict(X_test)

final_xgb_mse = mean_squared_error(y_test, y_pred_best_xgb)
print(f"Optimized XGBoost MSE: {final_xgb_mse}")


Optimized XGBoost MSE: 97129103568.81747


In [29]:
file_path = "Dataset-Capex.csv"
df = pd.read_csv(file_path)

df["Budget_Utilization"] = df["Actual_Expenditure"] / df["Allocated_Budget"]
df["Budget_Variance_Percent"] = (df["Variance"] / df["Allocated_Budget"]) * 100
df["Budget_Interaction"] = df["Allocated_Budget"] * df["Budget_Utilization"]
df["Expenditure_Interaction"] = df["Actual_Expenditure"] * df["Budget_Utilization"]

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_phase = encoder.fit_transform(df[["Project_Phase"]])
encoded_df = pd.DataFrame(encoded_phase, columns=encoder.get_feature_names_out(["Project_Phase"]))

X = pd.concat([df[["Allocated_Budget", "Actual_Expenditure", "Budget_Utilization",
                    "Budget_Variance_Percent", "Budget_Interaction", "Expenditure_Interaction"]],
               encoded_df], axis=1)
y = df["Variance"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
xgb_model = XGBRegressor(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"Mean Absolute Error: {mae_xgb:.2f}")
print(f"R² Score: {r2_xgb:.2f}")

Mean Absolute Error: 6103.36
R² Score: 0.93


In [ ]:
df.to_csv("CapEx_PredictionsFinal1.csv", index=False)

In [ ]:
df_test = pd.DataFrame(X_test, columns=X.columns)

# Convert predicted values to a Pandas Series and assign to df_test
df_test["Predicted_Variance"] = pd.Series(y_pred_xgb, index=df_test.index)
df_test.to_csv("CapEx_Predictions.csv", index=False)

In [ ]:
print("X_test shape:", X_test.shape)
print("y_pred_xgb shape:", y_pred_xgb.shape)

X_test shape: (20, 8)
y_pred_xgb shape: (20,)
